In [2]:
# Fast imports
import pandas as pd
import numpy as np
from pathlib import Path
import os

# Output path helper function
def get_output_path(filename):
    """Get the correct output path based on file type"""
    base_path = Path('../../04_outputs')
    
    # Processed data CSVs (merged/intermediate files)
    if any(x in filename.lower() for x in ['final', 'merged', 'processed', 'enhanced']):
        path = Path('../../01_data/processed') / filename
    # Predictions CSVs
    elif 'prediction' in filename.lower():
        path = base_path / 'predictions' / filename
    # Metrics CSVs
    elif any(x in filename.lower() for x in ['metric', 'summary', 'quantile', 'winkler', 'segment']):
        path = base_path / 'metrics' / filename
    # Models (pkl files)
    elif filename.endswith('.pkl'):
        path = Path('../../03_models/saved_models') / filename
    # JSON mappings
    elif filename.endswith('.json'):
        path = Path('../../03_models/model_artifacts') / filename
    # Visualizations (all image files and html)
    elif filename.endswith(('.png', '.jpg', '.jpeg', '.gif', '.svg', '.html')):
        path = base_path / 'visualizations' / filename
    # Default to processed data for other CSVs
    elif filename.endswith('.csv'):
        path = Path('../../01_data/processed') / filename
    else:
        path = base_path / filename
    
    # Create directory if it doesn't exist
    path.parent.mkdir(parents=True, exist_ok=True)
    return path

In [3]:
# Load data files
data_wink = pd.read_csv('../../01_data/processed/winkler_picp_by_part_dataset.csv')
# Standardize column name to PN
if 'Part_Number' in data_wink.columns:
    data_wink = data_wink.rename(columns={'Part_Number': 'PN'})

In [4]:
data_market = pd.read_csv('../../01_data/processed/FINAL_market_tightness_enhanced_data.csv')

In [5]:
data_pred = pd.read_csv('../../04_outputs/predictions/predictions_all_datasets.csv')
# Standardize column name to PN
if 'Part_Number' in data_pred.columns:
    data_pred = data_pred.rename(columns={'Part_Number': 'PN'})

In [6]:
data_wink

,PN,Dataset,Count,LGB_Winkler_80CI_Mean,LGB_Winkler_80CI_Std,LGB_PICP_80CI,LGB_Winkler_50CI_Mean,LGB_Winkler_50CI_Std,LGB_PICP_50CI,LGB_Q10,...,TF_Median_Q10_Q90,TF_Q25,TF_Q75,TF_Median_Q25_Q75,TF_MAE_Mean,TF_MAE_Std,LGB_PICP_80CI_Pct,LGB_PICP_50CI_Pct,TF_PICP_80CI_Pct,TF_PICP_50CI_Pct
0,019-012-001,Test,3,10.241008,0.327421,0.666667,6.184003,1.286605,0.666667,3.116407,...,10.724762,6.235512,11.695316,5.362094,3.323172,2.715165,66.666667,66.666667,66.666667,66.666667
1,019-012-001,Validation,3,9.491700,0.866306,1.000000,5.373644,0.383437,1.000000,3.302877,...,9.269037,4.558037,9.235289,4.629971,1.805873,1.072386,100.000000,100.000000,100.000000,66.666667
2,024147-000,Test,3,19.019381,1.296581,0.666667,11.848559,0.815136,0.333333,18.731756,...,21.533339,22.635690,32.850500,10.454515,9.120625,5.559306,66.666667,33.333333,66.666667,33.333333
3,024147-000,Validation,3,20.961379,1.002433,1.000000,11.954600,1.237402,0.666667,20.461531,...,19.847797,21.433070,31.050760,9.643410,5.007767,2.966043,100.000000,66.666667,100.000000,33.333333
4,066-01212-0102,Test,3,10.858780,0.655512,1.000000,6.551615,0.419563,0.333333,7.605503,...,14.343759,10.969621,18.080284,7.217339,4.570176,2.315185,100.000000,33.333333,66.666667,33.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,UA538728-1,Validation,3,5.635687,0.335711,0.666667,3.454261,0.576968,0.333333,1.071749,...,5.999056,2.022005,5.111835,3.080049,2.811931,1.518975,66.666667,33.333333,66.666667,33.333333
834,VG22-01,Test,3,12.352542,0.540428,0.666667,8.131578,1.289715,0.333333,8.060506,...,14.914103,12.312140,19.905184,7.502955,3.531612,2.457004,66.666667,33.333333,100.000000,66.666667
835,VG22-01,Validation,3,12.420664,0.467580,1.000000,7.383006,0.229920,0.666667,8.670539,...,13.355986,9.507976,16.240839,6.722719,2.320028,1.532033,100.000000,66.666667,100.000000,66.666667
836,Z014H000333B,Test,3,17.656081,0.566241,0.333333,14.027713,3.103627,0.333333,8.189821,...,18.881292,17.964209,26.951910,9.198866,9.561097,2.702143,33.333333,33.333333,66.666667,0.000000


In [7]:
data_market

,PN,year,month,Part Date,End User Companies,End User Inquiries,Non-End User Companies,Non-End User Inquiries,Total Sources,Total Quantity,...,relative_market_tightness_neu,is_market_squeeze_neu,stock_adjusted_tightness,avg_tightness_by_part,relative_market_tightness,is_market_squeeze,eu_demand_ratio,qty_per_source,date,tightness_category
0,019-012-001,2021,1,2021-01-01,10,12,33,60,77,236,...,0.997642,0,0.181435,0.551282,0.998189,0,0.227273,3.025641,2021-01-01,Loose (Buyers Abundant)
1,019-012-001,2021,2,2021-02-01,5,7,45,106,76,222,...,1.157840,0,0.224215,0.600316,1.079882,0,0.098039,2.883117,2021-02-01,Loose (Buyers Abundant)
2,019-012-001,2021,3,2021-03-01,4,5,15,43,74,216,...,0.495666,0,0.087558,0.484655,0.521632,0,0.200000,2.880000,2021-03-01,Loose (Buyers Abundant)
3,019-012-001,2021,4,2021-04-01,9,12,42,106,69,201,...,1.324874,0,0.252475,0.545634,1.332831,0,0.173077,2.871429,2021-04-01,Loose (Buyers Abundant)
4,019-012-001,2021,5,2021-05-01,6,7,33,88,71,198,...,1.009178,0,0.195980,0.544841,0.992353,0,0.150000,2.750000,2021-05-01,Loose (Buyers Abundant)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27197,Z014H000333B,2025,6,2025-06-01,20,25,136,320,35,83,...,1.321946,1,1.857143,3.251172,1.332443,1,0.127389,2.305556,2025-06-01,Tight (Scarce Supply)
27198,Z014H000333B,2025,7,2025-07-01,33,57,189,455,37,90,...,1.717296,1,2.439560,3.298280,1.770721,1,0.147982,2.368421,2025-07-01,Very Tight (Critical)
27199,Z014H000333B,2025,8,2025-08-01,33,46,359,821,37,80,...,3.135294,1,4.839506,3.423592,3.012268,1,0.083969,2.105263,2025-08-01,Very Tight (Critical)
27200,Z014H000333B,2025,9,2025-09-01,18,28,148,329,37,84,...,1.285937,1,1.952941,3.440168,1.269459,1,0.107784,2.210526,2025-09-01,Tight (Scarce Supply)


In [8]:
data_pred

,PN,Date,month_index,Actual_Demand,LightGBM_Prediction,Transformer_Prediction,LightGBM_Error,Transformer_Error,LightGBM_Absolute_Error,Transformer_Absolute_Error,Dataset,LGB_Q10,LGB_Q25,LGB_Q75,LGB_Q90,TF_Q10,TF_Q25,TF_Q75,TF_Q90
0,019-012-001,2022-05-01,17,11.0,8.131978,8.093681,2.868022,2.906319,2.868022,2.906319,Train,3.755664,4.998592,10.617832,13.069084,3.843694,5.745482,11.984263,15.561357
1,019-012-001,2022-06-01,18,3.0,2.248910,1.860687,0.751090,1.139313,0.751090,1.139313,Train,0.320452,1.056569,4.789926,6.024701,0.145882,0.890426,3.496274,5.149028
2,019-012-001,2022-07-01,19,4.0,2.214379,2.815168,1.785621,1.184832,1.785621,1.184832,Train,0.224317,0.930483,5.184620,5.864603,0.592271,1.582784,4.919506,6.908190
3,019-012-001,2022-08-01,20,15.0,9.637114,11.970062,5.362886,3.029938,5.362886,3.029938,Train,4.310878,6.126078,12.138282,15.176707,6.464181,8.928946,16.820787,21.226261
4,019-012-001,2022-09-01,21,8.0,9.297714,12.098417,-1.297714,-4.098417,1.297714,4.098417,Train,4.090499,6.540767,12.785694,16.045691,6.546081,9.047795,16.913055,21.314146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17174,VG22-01,2025-08-01,56,18.0,13.350911,14.835972,4.649089,3.164028,4.649089,3.164028,Test,7.607762,10.498490,16.596353,19.475954,8.726450,11.492960,19.894215,24.581348
17175,VG22-01,2025-09-01,57,24.0,13.007935,14.529065,10.992065,9.470937,10.992065,9.470937,Test,8.377709,11.180068,17.653865,20.292105,8.557051,11.250370,19.483227,24.093014
17176,Z014H000333B,2025-07-01,55,33.0,13.876164,18.244747,19.123836,14.755253,19.123836,14.755253,Test,7.281758,9.482143,18.166029,23.295142,11.320935,14.449413,23.992933,29.160130
17177,Z014H000333B,2025-08-01,56,33.0,14.594187,20.673857,18.405813,12.326143,18.405813,12.326143,Test,7.519625,10.024047,18.777652,22.983857,13.408630,16.687881,26.771507,32.119164


In [9]:
def clean_data(data_market):
    # Drop columns: 'IBA_fleet_count', 'IBA_age_mean' and 33 other columns
    data_market = data_market.drop(columns=['IBA_fleet_count', 'IBA_age_mean', 'IBA_age_median', 'IBA_age_min', 'IBA_age_max', 'IBA_build_year_min', 'IBA_build_year_max', 'IBA_build_year_mean', 'IBA_order_year_min', 'IBA_order_year_max', 'IBA_order_year_mean', 'IBA_lease_start_year_median', 'IBA_num_operators', 'IBA_num_owners', 'IBA_num_countries_operated_for', 'IBA_age_0_8_count', 'IBA_age_0_8_share', 'IBA_age_8_15_count', 'IBA_age_8_15_share', 'IBA_age_15_20_count', 'IBA_age_15_20_share', 'IBA_age_20_30_count', 'IBA_age_20_30_share', 'IBA_age_30_plus_count', 'IBA_age_30_plus_share', 'IBA_age_15_25_count', 'IBA_age_15_25_share', 'IBA_age_25_plus_count', 'IBA_age_25_plus_share', 'IBA_share_owned', 'IBA_share_leased', 'IBA_share_operating_lease', 'IBA_share_finance_lease', 'IBA_share_commercial', 'IBA_share_gov_mil'])
    return data_market

data_market_clean = clean_data(data_market.copy())
data_market_clean.head()

,PN,year,month,Part Date,End User Companies,End User Inquiries,Non-End User Companies,Non-End User Inquiries,Total Sources,Total Quantity,...,relative_market_tightness_neu,is_market_squeeze_neu,stock_adjusted_tightness,avg_tightness_by_part,relative_market_tightness,is_market_squeeze,eu_demand_ratio,qty_per_source,date,tightness_category
0,019-012-001,2021,1,2021-01-01,10,12,33,60,77,236,...,0.997642,0,0.181435,0.551282,0.998189,0,0.227273,3.025641,2021-01-01,Loose (Buyers Abundant)
1,019-012-001,2021,2,2021-02-01,5,7,45,106,76,222,...,1.157840,0,0.224215,0.600316,1.079882,0,0.098039,2.883117,2021-02-01,Loose (Buyers Abundant)
2,019-012-001,2021,3,2021-03-01,4,5,15,43,74,216,...,0.495666,0,0.087558,0.484655,0.521632,0,0.200000,2.880000,2021-03-01,Loose (Buyers Abundant)
3,019-012-001,2021,4,2021-04-01,9,12,42,106,69,201,...,1.324874,0,0.252475,0.545634,1.332831,0,0.173077,2.871429,2021-04-01,Loose (Buyers Abundant)
4,019-012-001,2021,5,2021-05-01,6,7,33,88,71,198,...,1.009178,0,0.195980,0.544841,0.992353,0,0.150000,2.750000,2021-05-01,Loose (Buyers Abundant)


In [10]:
# Check structure of each dataframe
print("=== DATA_WINK ===")
print(f"Shape: {data_wink.shape}")
print(f"Columns: {data_wink.columns.tolist()}")
print(f"\nSample data:")
print(data_wink.head(3))
print(f"\nDataset values: {data_wink['Dataset'].unique() if 'Dataset' in data_wink.columns else 'No Dataset column'}")

print("\n=== DATA_MARKET_CLEAN ===")
print(f"Shape: {data_market_clean.shape}")
print(f"Columns: {data_market_clean.columns.tolist()}")
print(f"\nSample data:")
print(data_market_clean.head(3))

print("\n=== DATA_PRED ===")
print(f"Shape: {data_pred.shape}")
print(f"Columns: {data_pred.columns.tolist()}")
print(f"\nSample data:")
print(data_pred.head(3))

=== DATA_WINK ===
Shape: (838, 35)
Columns: ['PN', 'Dataset', 'Count', 'LGB_Winkler_80CI_Mean', 'LGB_Winkler_80CI_Std', 'LGB_PICP_80CI', 'LGB_Winkler_50CI_Mean', 'LGB_Winkler_50CI_Std', 'LGB_PICP_50CI', 'LGB_Q10', 'LGB_Q90', 'LGB_Median_Q10_Q90', 'LGB_Q25', 'LGB_Q75', 'LGB_Median_Q25_Q75', 'LGB_MAE_Mean', 'LGB_MAE_Std', 'TF_Winkler_80CI_Mean', 'TF_Winkler_80CI_Std', 'TF_PICP_80CI', 'TF_Winkler_50CI_Mean', 'TF_Winkler_50CI_Std', 'TF_PICP_50CI', 'TF_Q10', 'TF_Q90', 'TF_Median_Q10_Q90', 'TF_Q25', 'TF_Q75', 'TF_Median_Q25_Q75', 'TF_MAE_Mean', 'TF_MAE_Std', 'LGB_PICP_80CI_Pct', 'LGB_PICP_50CI_Pct', 'TF_PICP_80CI_Pct', 'TF_PICP_50CI_Pct']

Sample data:
            PN     Dataset  Count  LGB_Winkler_80CI_Mean  \
0  019-012-001        Test      3              10.241008   
1  019-012-001  Validation      3               9.491700   
2   024147-000        Test      3              19.019381   

   LGB_Winkler_80CI_Std  LGB_PICP_80CI  LGB_Winkler_50CI_Mean  \
0              0.327421       0.666667 

In [11]:
# STEP 1: Calculate Winkler scores manually from prediction quantiles
# Instead of using external data_wink file, we calculate from data_pred
# Winkler score measures the quality of prediction intervals

def calculate_winkler_scores(df):
    """
    Calculate Winkler scores for 50% and 80% confidence intervals
    using prediction quantiles
    
    For a prediction interval [lower, upper] with coverage alpha:
    Winkler = width - (2/alpha) * penalty
    where penalty = max(0, lower-actual) + max(0, actual-upper)
    """
    
    df = df.copy()
    actual = df['Actual_Demand']
    
    # Only calculate for rows with actual demand
    mask = actual.notna()
    
    # 50% Confidence Interval: Q25 to Q75 (LightGBM)
    interval_width_50 = df.loc[mask, 'LGB_Q75'] - df.loc[mask, 'LGB_Q25']
    lower_50 = df.loc[mask, 'LGB_Q25']
    upper_50 = df.loc[mask, 'LGB_Q75']
    actual_mask = actual[mask]
    
    # Penalty for exceeding bounds
    penalty_50 = np.maximum(0, lower_50.values - actual_mask.values) + np.maximum(0, actual_mask.values - upper_50.values)
    
    # Winkler score (alpha = 0.5 for 50% CI)
    df.loc[mask, 'LGB_Winkler_50CI'] = interval_width_50.values - (2/0.5) * penalty_50
    
    # PICP: Prediction Interval Coverage Probability (% of actuals within interval)
    df.loc[mask, 'LGB_PICP_50CI'] = ((actual_mask >= lower_50.values) & (actual_mask <= upper_50.values)).astype(float).values
    
    # 80% Confidence Interval: Q10 to Q90 (LightGBM)
    interval_width_80 = df.loc[mask, 'LGB_Q90'] - df.loc[mask, 'LGB_Q10']
    lower_80 = df.loc[mask, 'LGB_Q10']
    upper_80 = df.loc[mask, 'LGB_Q90']
    
    # Penalty for exceeding bounds
    penalty_80 = np.maximum(0, lower_80.values - actual_mask.values) + np.maximum(0, actual_mask.values - upper_80.values)
    
    # Winkler score (alpha = 0.2 for 80% CI)
    df.loc[mask, 'LGB_Winkler_80CI'] = interval_width_80.values - (2/0.2) * penalty_80
    
    # PICP: Prediction Interval Coverage Probability
    df.loc[mask, 'LGB_PICP_80CI'] = ((actual_mask >= lower_80.values) & (actual_mask <= upper_80.values)).astype(float).values
    
    # MAE: Mean Absolute Error for LightGBM predictions
    df.loc[mask, 'LGB_MAE'] = np.abs(df.loc[mask, 'LightGBM_Prediction'].values - actual_mask.values)
    
    # Same calculations for Transformer model
    # 50% CI: Q25 to Q75 (Transformer)
    interval_width_50_tf = df.loc[mask, 'TF_Q75'] - df.loc[mask, 'TF_Q25']
    lower_50_tf = df.loc[mask, 'TF_Q25']
    upper_50_tf = df.loc[mask, 'TF_Q75']
    
    penalty_50_tf = np.maximum(0, lower_50_tf.values - actual_mask.values) + np.maximum(0, actual_mask.values - upper_50_tf.values)
    df.loc[mask, 'TF_Winkler_50CI'] = interval_width_50_tf.values - (2/0.5) * penalty_50_tf
    df.loc[mask, 'TF_PICP_50CI'] = ((actual_mask >= lower_50_tf.values) & (actual_mask <= upper_50_tf.values)).astype(float).values
    
    # 80% CI: Q10 to Q90 (Transformer)
    interval_width_80_tf = df.loc[mask, 'TF_Q90'] - df.loc[mask, 'TF_Q10']
    lower_80_tf = df.loc[mask, 'TF_Q10']
    upper_80_tf = df.loc[mask, 'TF_Q90']
    
    penalty_80_tf = np.maximum(0, lower_80_tf.values - actual_mask.values) + np.maximum(0, actual_mask.values - upper_80_tf.values)
    df.loc[mask, 'TF_Winkler_80CI'] = interval_width_80_tf.values - (2/0.2) * penalty_80_tf
    df.loc[mask, 'TF_PICP_80CI'] = ((actual_mask >= lower_80_tf.values) & (actual_mask <= upper_80_tf.values)).astype(float).values
    
    # MAE: Mean Absolute Error for Transformer predictions
    df.loc[mask, 'TF_MAE'] = np.abs(df.loc[mask, 'Transformer_Prediction'].values - actual_mask.values)
    
    return df

# Apply calculations to data_pred
data_pred_with_winkler = calculate_winkler_scores(data_pred)

print("✓ Winkler scores calculated manually from prediction quantiles")
print(f"\nNew Winkler columns added to data_pred:")
print(f"  • LGB_Winkler_50CI: Winkler score for 50% interval (LightGBM)")
print(f"  • LGB_Winkler_80CI: Winkler score for 80% interval (LightGBM)")
print(f"  • TF_Winkler_50CI: Winkler score for 50% interval (Transformer)")
print(f"  • TF_Winkler_80CI: Winkler score for 80% interval (Transformer)")
print(f"  • LGB_MAE / TF_MAE: Mean Absolute Error (point predictions)")
print(f"  • PICP_50CI / PICP_80CI: Coverage probabilities for both models")

print(f"\nSample of calculated Winkler scores:")
sample_winkler = data_pred_with_winkler[data_pred_with_winkler['Actual_Demand'].notna()][
    ['PN', 'Actual_Demand', 'LightGBM_Prediction', 'LGB_Q10', 'LGB_Q90', 
     'LGB_PICP_80CI', 'LGB_Winkler_80CI', 'LGB_MAE']].head(10)
print(sample_winkler.to_string())

# Replace data_pred with the enhanced version
data_pred = data_pred_with_winkler

✓ Winkler scores calculated manually from prediction quantiles

New Winkler columns added to data_pred:
  • LGB_Winkler_50CI: Winkler score for 50% interval (LightGBM)
  • LGB_Winkler_80CI: Winkler score for 80% interval (LightGBM)
  • TF_Winkler_50CI: Winkler score for 50% interval (Transformer)
  • TF_Winkler_80CI: Winkler score for 80% interval (Transformer)
  • LGB_MAE / TF_MAE: Mean Absolute Error (point predictions)
  • PICP_50CI / PICP_80CI: Coverage probabilities for both models

Sample of calculated Winkler scores:
            PN  Actual_Demand  LightGBM_Prediction   LGB_Q10    LGB_Q90  LGB_PICP_80CI  LGB_Winkler_80CI   LGB_MAE
0  019-012-001           11.0             8.131978  3.755664  13.069084            1.0          9.313420  2.868022
1  019-012-001            3.0             2.248910  0.320452   6.024701            1.0          5.704248  0.751090
2  019-012-001            4.0             2.214379  0.224317   5.864603            1.0          5.640285  1.785621
3  019-012

In [12]:
# STEP 1b: Merge market data with prediction data (which now includes calculated Winkler scores)
# No need for separate data_wink merge - Winkler scores are now part of data_pred

# First create month_index in data_market_clean
# IMPORTANT: Use the same base_date as predictions_all_datasets.csv
# Predictions use: month_index 17 = 2022-05-01, so base_date = 2020-12-01
data_market_clean['date'] = pd.to_datetime(data_market_clean['date'])
base_date = pd.Timestamp('2020-12-01')
data_market_clean['month_index'] = ((data_market_clean['date'].dt.year - base_date.year) * 12 + 
                                      (data_market_clean['date'].dt.month - base_date.month))

# Merge market data with prediction data (which now contains Winkler scores)
# Important: data_pred also has month_index calculated from dates, merge on both PN and month_index
final_merged = data_market_clean.merge(
    data_pred,
    left_on=['PN', 'month_index'],
    right_on=['PN', 'month_index'],
    how='left',
    suffixes=('', '_pred')
)

# Rename Dataset column to Dataset_pred for clarity
if 'Dataset' in final_merged.columns:
    final_merged.rename(columns={'Dataset': 'Dataset_pred'}, inplace=True)

print(f"✓ Market data merged with prediction data (with calculated Winkler scores)")
print(f"  • Base date used: {base_date} (same as predictions_all_datasets.csv)")
print(f"  • Final merged shape: {final_merged.shape}")
print(f"  • Rows with predictions & Winkler scores: {final_merged['Actual_Demand'].notna().sum():,}")
print(f"  • Merge verification:")
print(f"    - Unique PNs in market data: {data_market_clean['PN'].nunique()}")
print(f"    - Unique Part_Numbers in pred data: {data_pred['PN'].nunique()}")
print(f"    - Matches with predictions: {final_merged['Actual_Demand'].notna().sum():,} out of {len(final_merged):,}")

✓ Market data merged with prediction data (with calculated Winkler scores)
  • Base date used: 2020-12-01 00:00:00 (same as predictions_all_datasets.csv)
  • Final merged shape: (27202, 69)
  • Rows with predictions & Winkler scores: 17,179
  • Merge verification:
    - Unique PNs in market data: 469
    - Unique Part_Numbers in pred data: 419
    - Matches with predictions: 17,179 out of 27,202


In [13]:
# Verification of merged data
print("✓ Data merge complete (Winkler scores calculated and included)")
print(f"\nMerged dataset structure:")
print(f"  • Total rows: {final_merged.shape[0]:,}")
print(f"  • Total columns: {final_merged.shape[1]}")
print(f"  • Rows with Winkler scores & predictions: {final_merged['Actual_Demand'].notna().sum():,}")

print(f"\nWinkler metrics now available:")
winkler_metrics = [col for col in final_merged.columns if 'Winkler' in col or 'PICP' in col or 'MAE' in col]
for col in sorted(winkler_metrics):
    print(f"  ✓ {col}: {final_merged[col].notna().sum():,} values")


✓ Data merge complete (Winkler scores calculated and included)

Merged dataset structure:
  • Total rows: 27,202
  • Total columns: 69
  • Rows with Winkler scores & predictions: 17,179

Winkler metrics now available:
  ✓ LGB_MAE: 17,179 values
  ✓ LGB_PICP_50CI: 17,179 values
  ✓ LGB_PICP_80CI: 17,179 values
  ✓ LGB_Winkler_50CI: 17,179 values
  ✓ LGB_Winkler_80CI: 17,179 values
  ✓ TF_MAE: 17,179 values
  ✓ TF_PICP_50CI: 17,179 values
  ✓ TF_PICP_80CI: 17,179 values
  ✓ TF_Winkler_50CI: 17,179 values
  ✓ TF_Winkler_80CI: 17,179 values


In [14]:
# Check alignment of dates and month_index
print("Date to month_index mapping verification:")
date_mapping = final_merged[['date', 'month_index']].drop_duplicates().sort_values('month_index')
print(date_mapping.head(20).to_string())

print(f"\nDate range: {final_merged['date'].min()} to {final_merged['date'].max()}")
print(f"Month index range: {final_merged['month_index'].min()} to {final_merged['month_index'].max()}")


Date to month_index mapping verification:
         date  month_index
0  2021-01-01            1
1  2021-02-01            2
2  2021-03-01            3
3  2021-04-01            4
4  2021-05-01            5
5  2021-06-01            6
6  2021-07-01            7
7  2021-08-01            8
8  2021-09-01            9
9  2021-10-01           10
10 2021-11-01           11
11 2021-12-01           12
12 2022-01-01           13
13 2022-02-01           14
14 2022-03-01           15
15 2022-04-01           16
16 2022-05-01           17
17 2022-06-01           18
18 2022-07-01           19
19 2022-08-01           20

Date range: 2021-01-01 00:00:00 to 2025-10-01 00:00:00
Month index range: 1 to 58


In [15]:
# Summary of final merged dataset with calculated Winkler scores
print("="*80)
print("FINAL MERGED DATASET WITH CALCULATED WINKLER SCORES")
print("="*80)

print(f"\n📊 DATASET DIMENSIONS:")
print(f"  • Total rows: {final_merged.shape[0]:,}")
print(f"  • Total columns: {final_merged.shape[1]}")
print(f"  • Unique parts: {final_merged['PN'].nunique()}")
print(f"  • Date range: {final_merged['date'].min()} to {final_merged['date'].max()}")

print(f"\n🔗 MERGE RESULTS:")
print(f"  • Rows with calculated Winkler scores: {final_merged['LGB_Winkler_80CI'].notna().sum():,}")
print(f"  • Rows with Actual Demand: {final_merged['Actual_Demand'].notna().sum():,}")
print(f"  • Coverage for 50% CI (Test set): {final_merged[final_merged['Dataset_pred']=='Test']['LGB_PICP_50CI'].mean():.1%}")
print(f"  • Coverage for 80% CI (Test set): {final_merged[final_merged['Dataset_pred']=='Test']['LGB_PICP_80CI'].mean():.1%}")

print(f"\n📋 COLUMN CATEGORIES:")
print(f"  • Market tightness columns: {len([c for c in final_merged.columns if c in data_market_clean.columns])}")
print(f"  • Calculated Winkler metrics: {len([c for c in final_merged.columns if 'Winkler' in c or 'PICP' in c])}")
print(f"  • Prediction columns: 6 (LGB & TF predictions + quantiles)")

print(f"\n✅ DATA INTEGRATION COMPLETE!")
print(f"  The 'final_merged' dataframe contains:")
print(f"  1. All market tightness metrics (EU, NEU, combined)")
print(f"  2. Winkler scores calculated manually from prediction quantiles")
print(f"  3. Prediction intervals (Q10, Q25, Q75, Q90)")
print(f"  4. Actual demand and predictions from all datasets (Train/Val/Test)")


FINAL MERGED DATASET WITH CALCULATED WINKLER SCORES

📊 DATASET DIMENSIONS:
  • Total rows: 27,202
  • Total columns: 69
  • Unique parts: 469
  • Date range: 2021-01-01 00:00:00 to 2025-10-01 00:00:00

🔗 MERGE RESULTS:
  • Rows with calculated Winkler scores: 17,179
  • Rows with Actual Demand: 17,179
  • Coverage for 50% CI (Test set): 50.3%
  • Coverage for 80% CI (Test set): 78.8%

📋 COLUMN CATEGORIES:
  • Market tightness columns: 42
  • Calculated Winkler metrics: 8
  • Prediction columns: 6 (LGB & TF predictions + quantiles)

✅ DATA INTEGRATION COMPLETE!
  The 'final_merged' dataframe contains:
  1. All market tightness metrics (EU, NEU, combined)
  2. Winkler scores calculated manually from prediction quantiles
  3. Prediction intervals (Q10, Q25, Q75, Q90)
  4. Actual demand and predictions from all datasets (Train/Val/Test)


In [16]:
# Summary of the merged dataset
print("="*70)
print("FINAL MERGED DATASET SUMMARY")
print("="*70)

print(f"\n📊 DATASET DIMENSIONS:")
print(f"  • Total rows: {final_merged.shape[0]:,}")
print(f"  • Total columns: {final_merged.shape[1]}")
print(f"  • Unique parts: {final_merged['PN'].nunique()}")
print(f"  • Date range: {final_merged['date'].min()} to {final_merged['date'].max()}")

print(f"\n🔗 MERGE RESULTS:")
print(f"  • Rows with Winkler data (Test): {final_merged['LGB_Winkler_80CI'].notna().sum():,}")
print(f"  • Rows with Prediction data: {final_merged['Actual_Demand'].notna().sum():,}")
print(f"  • Rows with both Winkler & Predictions: {((final_merged['LGB_Winkler_80CI'].notna()) & (final_merged['Actual_Demand'].notna())).sum():,}")

print(f"\n📋 COLUMN CATEGORIES:")
print(f"  • Original market_tightness columns: {len([c for c in final_merged.columns if c in data_market_clean.columns])}")
print(f"  • Winkler metrics columns: {len([c for c in final_merged.columns if c in data_wink.columns and c != 'PN'])}")
print(f"  • Prediction columns: {len([c for c in data_pred.columns if c not in ['PN', 'month_index']])}")

print(f"\n✅ DATA INTEGRATION COMPLETE!")
print(f"  The 'final_merged' dataframe contains:")
print(f"  1. All market tightness metrics from data_market_clean")
print(f"  2. Winkler scores from data_wink (Test dataset only)")
print(f"  3. Predictions from data_pred (matched by Part Number and month)")

# Display column list
print(f"\n📝 All columns in final_merged:")
for i, col in enumerate(final_merged.columns, 1):
    print(f"  {i:2d}. {col}")
    
final_merged.head()

FINAL MERGED DATASET SUMMARY

📊 DATASET DIMENSIONS:
  • Total rows: 27,202
  • Total columns: 69
  • Unique parts: 469
  • Date range: 2021-01-01 00:00:00 to 2025-10-01 00:00:00

🔗 MERGE RESULTS:
  • Rows with Winkler data (Test): 17,179
  • Rows with Prediction data: 17,179
  • Rows with both Winkler & Predictions: 17,179

📋 COLUMN CATEGORIES:
  • Original market_tightness columns: 42
  • Winkler metrics columns: 12
  • Prediction columns: 27

✅ DATA INTEGRATION COMPLETE!
  The 'final_merged' dataframe contains:
  1. All market tightness metrics from data_market_clean
  2. Winkler scores from data_wink (Test dataset only)
  3. Predictions from data_pred (matched by Part Number and month)

📝 All columns in final_merged:
   1. PN
   2. year
   3. month
   4. Part Date
   5. End User Companies
   6. End User Inquiries
   7. Non-End User Companies
   8. Non-End User Inquiries
   9. Total Sources
  10. Total Quantity
  11. Quantity NE
  12. Quantity NS
  13. Quantity OH
  14. Quantity SV
 

,PN,year,month,Part Date,End User Companies,End User Inquiries,Non-End User Companies,Non-End User Inquiries,Total Sources,Total Quantity,...,LGB_Winkler_50CI,LGB_PICP_50CI,LGB_Winkler_80CI,LGB_PICP_80CI,LGB_MAE,TF_Winkler_50CI,TF_PICP_50CI,TF_Winkler_80CI,TF_PICP_80CI,TF_MAE
0,019-012-001,2021,1,2021-01-01,10,12,33,60,77,236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,019-012-001,2021,2,2021-02-01,5,7,45,106,76,222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,019-012-001,2021,3,2021-03-01,4,5,15,43,74,216,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,019-012-001,2021,4,2021-04-01,9,12,42,106,69,201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,019-012-001,2021,5,2021-05-01,6,7,33,88,71,198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Optional: Keep Winkler scores for all datasets since they're calculated from predictions
# But note that they're most reliable for Test dataset predictions

print("✓ Winkler scores calculated for all prediction datasets")
print(f"\nWinkler score availability by dataset:")

datasets = ['Train', 'Validation', 'Test']
for ds in datasets:
    count = (final_merged['Dataset_pred'] == ds).sum()
    winkler_count = (final_merged['Dataset_pred'] == ds) & (final_merged['LGB_Winkler_80CI'].notna())
    print(f"  • {ds}: {winkler_count.sum():,} rows with scores out of {count:,} total")

print(f"\n📌 Note: Winkler scores are calculated from prediction quantiles")
print(f"  • Most reliable for: Test dataset (used for validation)")
print(f"  • Also available for: Train & Validation datasets")
print(f"  • Metric explanation:")
print(f"    - PICP: % of actual values within prediction interval (target: 50% or 80%)")
print(f"    - Winkler: Interval width - penalty for actual values outside interval")
print(f"    - MAE: Mean Absolute Error of point prediction")


✓ Winkler scores calculated for all prediction datasets

Winkler score availability by dataset:
  • Train: 14,665 rows with scores out of 14,665 total
  • Validation: 1,257 rows with scores out of 1,257 total
  • Test: 1,257 rows with scores out of 1,257 total

📌 Note: Winkler scores are calculated from prediction quantiles
  • Most reliable for: Test dataset (used for validation)
  • Also available for: Train & Validation datasets
  • Metric explanation:
    - PICP: % of actual values within prediction interval (target: 50% or 80%)
    - Winkler: Interval width - penalty for actual values outside interval
    - MAE: Mean Absolute Error of point prediction


In [18]:
# Check column names related to Dataset
dataset_cols = [col for col in final_merged.columns if 'dataset' in col.lower() or 'Dataset' in col]
print("Dataset-related columns:", dataset_cols)
print("\nAll columns containing 'pred':")
pred_related = [col for col in final_merged.columns if 'pred' in col.lower()]
print(pred_related)

Dataset-related columns: ['Dataset_pred']

All columns containing 'pred':
['LightGBM_Prediction', 'Transformer_Prediction', 'Dataset_pred']


In [19]:
final_merged

,PN,year,month,Part Date,End User Companies,End User Inquiries,Non-End User Companies,Non-End User Inquiries,Total Sources,Total Quantity,...,LGB_Winkler_50CI,LGB_PICP_50CI,LGB_Winkler_80CI,LGB_PICP_80CI,LGB_MAE,TF_Winkler_50CI,TF_PICP_50CI,TF_Winkler_80CI,TF_PICP_80CI,TF_MAE
0,019-012-001,2021,1,2021-01-01,10,12,33,60,77,236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,019-012-001,2021,2,2021-02-01,5,7,45,106,76,222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,019-012-001,2021,3,2021-03-01,4,5,15,43,74,216,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,019-012-001,2021,4,2021-04-01,9,12,42,106,69,201,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,019-012-001,2021,5,2021-05-01,6,7,33,88,71,198,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27197,Z014H000333B,2025,6,2025-06-01,20,25,136,320,35,83,...,-2.204224,0.0,15.371583,1.0,6.656874,-1.156807,0.0,14.996149,1.0,7.115805
27198,Z014H000333B,2025,7,2025-07-01,33,57,189,455,37,90,...,-50.651999,0.0,-81.035198,0.0,19.123836,-26.484748,0.0,-20.559505,0.0,14.755253
27199,Z014H000333B,2025,8,2025-08-01,33,46,359,821,37,80,...,-48.135786,0.0,-84.697194,0.0,18.405813,-14.830346,0.0,9.902174,0.0,12.326143
27200,Z014H000333B,2025,9,2025-09-01,18,28,148,329,37,84,...,8.824140,1.0,16.704738,1.0,3.080696,7.042760,0.0,19.592047,1.0,5.086348


In [20]:
# Display a focused view of key merged columns
key_columns = [
    'PN', 'Demand_Type', 'Dataset_pred', 'date', 'month_index',
    # Market tightness metrics
    'End User Companies', 'Non-End User Companies', 'Total Sources', 'tightness_category', 
    'market_tightness_eu','stock_adjusted_tightness_eu','avg_tightness_by_part_eu','is_market_squeeze_eu',
    'market_tightness_neu', 'stock_adjusted_tightness_neu', 'avg_tightness_by_part_neu', 'is_market_squeeze_neu',
    
    # Predictions
    'Actual_Demand', 'LightGBM_Prediction', 'Transformer_Prediction',
    # Prediction quantiles
    'LGB_Q10', 'LGB_Q25', 'LGB_Q75', 'LGB_Q90', 'TF_Q10', 'TF_Q25', 'TF_Q75', 'TF_Q90',
    
    # Winkler scores (individual row scores, not aggregated means)
    'LGB_PICP_50CI', 'LGB_Winkler_50CI', 'LGB_PICP_80CI', 'LGB_Winkler_80CI', 'LGB_MAE',
    'TF_PICP_50CI', 'TF_Winkler_50CI', 'TF_PICP_80CI', 'TF_Winkler_80CI', 'TF_MAE',
]

# Show only rows that have prediction data for better visibility
sample_with_predictions = final_merged[final_merged['Actual_Demand'].notna()][key_columns]#.head(15)
sample_with_predictions = sample_with_predictions.astype({'Actual_Demand': 'int'})
print("Sample of fully merged data (rows with predictions, Winkler scores calculated from quantiles):")
print("="*120)
sample_with_predictions


Sample of fully merged data (rows with predictions, Winkler scores calculated from quantiles):


,PN,Demand_Type,Dataset_pred,date,month_index,End User Companies,Non-End User Companies,Total Sources,tightness_category,market_tightness_eu,...,LGB_PICP_50CI,LGB_Winkler_50CI,LGB_PICP_80CI,LGB_Winkler_80CI,LGB_MAE,TF_PICP_50CI,TF_Winkler_50CI,TF_PICP_80CI,TF_Winkler_80CI,TF_MAE
16,019-012-001,Erratic,Train,2022-05-01,17,11,49,69,Loose (Buyers Abundant),0.157143,...,0.0,4.090570,1.0,9.313420,2.868022,1.0,6.238781,1.0,11.717664,2.906319
17,019-012-001,Erratic,Train,2022-06-01,18,3,33,70,Loose (Buyers Abundant),0.042254,...,1.0,3.733356,1.0,5.704248,0.751090,1.0,2.605848,1.0,5.003146,1.139313
18,019-012-001,Erratic,Train,2022-07-01,19,4,64,68,Loose (Buyers Abundant),0.057971,...,1.0,4.254137,1.0,5.640285,1.785621,1.0,3.336722,1.0,6.315919,1.184832
19,019-012-001,Erratic,Train,2022-08-01,20,15,57,67,Moderate (Balanced),0.220588,...,0.0,-5.434666,1.0,10.865829,5.362886,1.0,7.891841,1.0,14.762080,3.029938
20,019-012-001,Erratic,Train,2022-09-01,21,8,21,67,Loose (Buyers Abundant),0.117647,...,1.0,6.244927,1.0,11.955191,1.297714,0.0,3.674080,1.0,14.768065,4.098417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27196,Z014H000333B,Smooth,Validation,2025-05-01,53,21,166,33,Very Tight (Critical),0.617647,...,0.0,1.621146,1.0,14.843988,6.199366,0.0,-6.029569,1.0,14.673198,8.160976
27197,Z014H000333B,Smooth,Validation,2025-06-01,54,20,136,35,Tight (Scarce Supply),0.555556,...,0.0,-2.204224,1.0,15.371583,6.656874,0.0,-1.156807,1.0,14.996149,7.115805
27198,Z014H000333B,Smooth,Test,2025-07-01,55,33,189,37,Very Tight (Critical),0.868421,...,0.0,-50.651999,0.0,-81.035198,19.123836,0.0,-26.484748,0.0,-20.559505,14.755253
27199,Z014H000333B,Smooth,Test,2025-08-01,56,33,359,37,Very Tight (Critical),0.868421,...,0.0,-48.135786,0.0,-84.697194,18.405813,0.0,-14.830346,0.0,9.902174,12.326143


In [21]:
# Save final merged dataset to processed data folder
output_path = get_output_path('FINAL_final_merged_with_winkler_scores.csv')
final_merged.to_csv(output_path, index=False)
print(f"✓ Saved final merged dataset to: {output_path}")

✓ Saved final merged dataset to: ..\..\01_data\processed\FINAL_final_merged_with_winkler_scores.csv
